In [ ]:
! pip install opencv-python==3.4.2.17

! pip install opencv-contrib-python==3.4.2.17

In [ ]:

import random
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Research/Thesis (4 - II)/Classification/Data/Jahid/Data.zip', 'r')
file = zip_ref.extractall("./tmp")
zip_ref.close()

print("o")

In [ ]:
DATASET="/content/tmp"


In [ ]:
CATEGORIES = ['CNV', 'DME', 'DRUSEN', 'NORMAL']
        
train_data=[]
sift_data = []

# `SIFT`

In [ ]:
from skimage import feature

def sift_image(image):
	# create SIFT feature extractor
    sift = cv2.xfeatures2d.SIFT_create()
    # detect features from the image
    keypoints, descriptors = sift.detectAndCompute(image, None)
    # draw the detected key points
    sift_image = cv2.drawKeypoints(image, keypoints, image)

    return sift_image
    

In [ ]:
import os
import tensorflow as tf

import cv2

for category in CATEGORIES:
        label=CATEGORIES.index(category)
        path=os.path.join(DATASET,category)
        for img_file in os.listdir(path):
            img=cv2.imread(os.path.join(path,img_file),1)
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img=cv2.resize(img,(224,224))
            sift_img =  sift_image(img)           
            train_data.append([img,label])
            sift_data.append([sift_img,label])
            

print(len(train_data))
print(len(sift_data))

In [ ]:
random.shuffle(sift_data)
random.shuffle(sift_data)
random.shuffle(sift_data)
random.shuffle(sift_data)

In [ ]:
for lbl in sift_data[:10]:
    print(lbl[1])

In [ ]:
X, y = [], []
for features,label in sift_data:
    X.append(features)
    y.append(label)


In [ ]:
len(X)

In [ ]:
# plt.imshow(hog_data[0][0])

In [ ]:
X_arr = np.array(X)
X_arr = X_arr.reshape((-1, 224*224*3))
y_arr = np.array(y)
X_arr.shape, y_arr.shape

In [ ]:
Xtrain, Xtest, ytrain, ytest  = train_test_split(X_arr, y_arr, test_size=0.2, shuffle=True, random_state=9990)


In [ ]:
Xtrain.shape

# SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

model_svm = SVC(kernel='rbf', class_weight='balanced')
model_svm.fit(Xtrain, ytrain)

In [ ]:
model = model_svm

In [ ]:
# model = grid.best_estimator_
yfit = model.predict(Xtest)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest, yfit,
                            target_names=CATEGORIES))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
mat = confusion_matrix(ytest, yfit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=CATEGORIES,
            yticklabels=CATEGORIES)
plt.xlabel('true label')
plt.ylabel('predicted label');

In [ ]:
import pickle

a = {'hello': 'world'}

with open('/content/drive/MyDrive/sift_svm.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)


# `PCA With SVM` 

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline

pca = RandomizedPCA(n_components=150, whiten=True, random_state=42)
svc = SVC(kernel='rbf', class_weight='balanced')
model_pca_svm = make_pipeline(pca, svc)

In [ ]:
param_grid = {'svc__C': [50],
              'svc__gamma': [0.005]}
grid = GridSearchCV(model_pca_svm, param_grid)

%time grid.fit(Xtrain, ytrain)
print(grid.best_params_)

In [ ]:
model = model_pca_svm

In [ ]:
model = grid.best_estimator_
yfit = model.predict(Xtest)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest, yfit,
                            target_names=CATEGORIES))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
mat = confusion_matrix(ytest, yfit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=CATEGORIES,
            yticklabels=CATEGORIES)
plt.xlabel('true label')
plt.ylabel('predicted label');